In [1]:
from dataclasses import dataclass
import os
from typing import Optional, Literal, Union, List

@dataclass
class TrainingConfig:
    batch_size: int = 64  # the batch size
    mega_batch: int = 1000 # how many batches to use for batchsampling
    num_epochs: int = 2  # the number of epochs to train the model
    gradient_accumulation_steps: int = 2  # the number of steps to accumulate gradients before taking an optimizer step
    learning_rate: float = 1e-4  # the learning rate
    lr_warmup_steps:int  = 1000
    save_image_steps:int  = 10000
    save_model_steps: int = 10000
    mixed_precision: str = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    optimizer: str = "AdamW"  # the optimizer to use, choose between `AdamW`, `Adam`, `SGD`, and `Adamax`
    SGDmomentum: float = 0.9
    output_dir: str = os.path.join("output","protein-VAE-UniRef50-5")  # the model name locally and on the HF Hub
    pad_to_multiple_of: int = 16
    max_len: int = 512  # truncation of the input sequence

    class_embeddings_concat = False  # whether to concatenate the class embeddings to the time embeddings

    push_to_hub = False  # whether to upload the saved model to the HF Hub
    hub_model_id = "kkj15dk/protein-VAE"  # the name of the repository to create on the HF Hub
    hub_private_repo = False
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed: int = 42

    labels_file = 'labels_test.json'

    cutoff: Optional[float] = None # cutoff for when to predict the token given the logits, and when to assign the unknown token 'X' to this position
    skip_special_tokens = False # whether to skip the special tokens when writing the evaluation sequences
    kl_weight: float = 0.05 # the weight of the KL divergence in the loss function

    weight_decay: float = 0.01 # weight decay for the optimizer
    grokfast: bool = False # whether to use the grokfast algorithm
    grokfast_alpha: float = 0.98 #Momentum hyperparmeter of the EMA.
    grokfast_lamb: float = 2.0 #Amplifying factor hyperparameter of the filter.

config = TrainingConfig()
print(vars(config))

{'batch_size': 64, 'mega_batch': 1000, 'num_epochs': 2, 'gradient_accumulation_steps': 2, 'learning_rate': 0.0001, 'lr_warmup_steps': 1000, 'save_image_steps': 10000, 'save_model_steps': 10000, 'mixed_precision': 'fp16', 'optimizer': 'AdamW', 'SGDmomentum': 0.9, 'output_dir': 'output/protein-VAE-UniRef50-5', 'pad_to_multiple_of': 16, 'max_len': 512, 'seed': 42, 'cutoff': None, 'kl_weight': 0.05, 'weight_decay': 0.01, 'grokfast': False, 'grokfast_alpha': 0.98, 'grokfast_lamb': 2.0}


In [2]:
from datasets import load_dataset

# # config.dataset_name = "kkj15dk/test_dataset"
# config.dataset_name = "agemagician/uniref50"
# # config.dataset_name = "kkj15dk/UniRef50-encoded"
# dataset = load_dataset(config.dataset_name) # , download_mode='force_redownload')
# dataset = dataset.shuffle(config.seed)

In [3]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("kkj15dk/protein_tokenizer")

def encode(example):
    return tokenizer(example['text'],
    # return tokenizer(example['sequence'],
                    padding = True,
                    pad_to_multiple_of = config.pad_to_multiple_of,
                    return_token_type_ids=False,
                    return_attention_mask=False, # We need to attend to padding tokens, so we set this to False
)
# dataset_train = dataset['train'].map(encode, batched=False, remove_columns=["sequence"])
# dataset_test = dataset['test'].map(encode, batched=False, remove_columns=["sequence"])
# dataset_val = dataset['val'].map(encode, batched=False, remove_columns=["sequence"])

In [4]:
# dataset_train = dataset['train']
# dataset_test = dataset['test']
# dataset_val = dataset['val']

from datasets import load_from_disk

datasets_folder = 'datasets'

train_file = os.path.join(datasets_folder, 'train_dataset')
test_file = os.path.join(datasets_folder, 'test_dataset')
val_file = os.path.join(datasets_folder, 'val_dataset')

if os.path.exists(train_file):
    print('loading train_dataset')
    dataset_train = load_from_disk(train_file)
else:
    print('encoding train_dataset')
    dataset_train = dataset['train'].map(encode, batched=False, remove_columns=["text"], num_proc=os.cpu_count())
    dataset_train.save_to_disk(train_file)

if os.path.exists(test_file):
    print('loading test_dataset')
    dataset_test = load_from_disk(test_file)
else:
    print('encoding test_dataset')
    dataset_test = dataset['test'].map(encode, batched=False, remove_columns=["text"], num_proc=os.cpu_count())
    dataset_test.save_to_disk(test_file)

if os.path.exists(val_file):
    print('loading val_dataset')
    dataset_val = load_from_disk(val_file)
else:
    print('encoding val_dataset')
    dataset_val = dataset['validation'].map(encode, batched=False, remove_columns=["text"], num_proc=os.cpu_count())
    dataset_val.save_to_disk(val_file)


loading train_dataset


Loading dataset from disk:   0%|          | 0/143 [00:00<?, ?it/s]

loading test_dataset
loading val_dataset


In [5]:
print(dataset_val)
print(dataset_val[0])
print(dataset_val[0]['input_ids'])
print(dataset_val[0]['name'])
print(len(dataset_val[0]['input_ids']))
print(tokenizer.decode(dataset_val[0]['input_ids'], skip_special_tokens=False))
print(tokenizer.decode(dataset_val[0]['input_ids'], skip_special_tokens=True))

Dataset({
    features: ['id', 'name', 'input_ids'],
    num_rows: 6084
})
{'id': 4909, 'name': 'UniRef50_E0S3A2', 'input_ids': [0, 13, 18, 12, 14, 6, 12, 8, 11, 5, 18, 16, 3, 10, 20, 6, 11, 12, 19, 8, 5, 6, 17, 7, 13, 18, 17, 10, 19, 18, 10, 8, 12, 19, 15, 8, 4, 18, 10, 17, 20, 10, 17, 14, 5, 11, 14, 17, 15, 13, 12, 10, 22, 18, 17, 5, 19, 10, 10, 3, 12, 14, 17, 14, 6, 4, 11, 8, 10, 6, 20, 3, 6, 20, 3, 8, 1, 23, 23, 23]}
[0, 13, 18, 12, 14, 6, 12, 8, 11, 5, 18, 16, 3, 10, 20, 6, 11, 12, 19, 8, 5, 6, 17, 7, 13, 18, 17, 10, 19, 18, 10, 8, 12, 19, 15, 8, 4, 18, 10, 17, 20, 10, 17, 14, 5, 11, 14, 17, 15, 13, 12, 10, 22, 18, 17, 5, 19, 10, 10, 3, 12, 14, 17, 14, 6, 4, 11, 8, 10, 6, 20, 3, 6, 20, 3, 8, 1, 23, 23, 23]
UniRef50_E0S3A2
80
[MSLNELGKDSQAIVEKLTGDERFMSRITSIGLTPGCSIRVIRNDKNRPMLIYSRDTIIALNRNECKGIEVAEVAG]---
MSLNELGKDSQAIVEKLTGDERFMSRITSIGLTPGCSIRVIRNDKNRPMLIYSRDTIIALNRNECKGIEVAEVAG


In [6]:
import torch
import torch.nn.functional as F

input_ids_tensor = torch.tensor(dataset_val[0]['input_ids'], dtype=torch.long)
onehot_seq = F.one_hot(input_ids_tensor, num_classes=tokenizer.vocab_size + 1).permute(1, 0).unsqueeze(0)
print(onehot_seq.shape)

def logits_to_token_ids(tokenizer, logits, cutoff = None):
        '''
        Convert a batch of logits to token_ids.
        Returns token_ids
        '''
        if cutoff is None:
            token_ids = logits.argmax(dim=1)
        else:
            token_ids = torch.where(logits.max(dim=1).values > cutoff, 
                                    logits.argmax(dim=1), 
                                    torch.tensor([tokenizer.unknown_token_id])
                                    )

        return token_ids
token_ids = logits_to_token_ids(tokenizer, onehot_seq)
print(token_ids)
print(tokenizer.batch_decode(token_ids, skip_special_tokens=config.skip_special_tokens))


torch.Size([1, 24, 80])
tensor([[ 0, 13, 18, 12, 14,  6, 12,  8, 11,  5, 18, 16,  3, 10, 20,  6, 11, 12,
         19,  8,  5,  6, 17,  7, 13, 18, 17, 10, 19, 18, 10,  8, 12, 19, 15,  8,
          4, 18, 10, 17, 20, 10, 17, 14,  5, 11, 14, 17, 15, 13, 12, 10, 22, 18,
         17,  5, 19, 10, 10,  3, 12, 14, 17, 14,  6,  4, 11,  8, 10,  6, 20,  3,
          6, 20,  3,  8,  1, 23, 23, 23]])
['[MSLNELGKDSQAIVEKLTGDERFMSRITSIGLTPGCSIRVIRNDKNRPMLIYSRDTIIALNRNECKGIEVAEVAG]---']


In [7]:
import random

def collate_fn(batch): # Can definitely be optimized
    max_len = max(len(x['input_ids']) for x in batch)
    if max_len > config.max_len:
        max_len = config.max_len
    input_ids = torch.zeros(len(batch), max_len, dtype=torch.long)
    attention_mask = torch.zeros(len(batch), max_len, dtype=torch.float)
    class_labels = torch.zeros(len(batch), dtype=torch.long)
    # identifiers = [x['id'] for x in batch]
    identifiers = [x.get('name', 'N/A') for x in batch]
    for i, x in enumerate(batch):
        seq_len = len(x['input_ids'])
        if seq_len > max_len:
            index = random.randint(0, seq_len - max_len)
            x['input_ids'] = x['input_ids'][index:index+max_len]
            seq_len = max_len
        input_ids[i, :seq_len] = torch.tensor(x['input_ids'], dtype=torch.long)
        attention_mask[i, :seq_len] = torch.tensor(1, dtype=torch.float)
        # class_labels[i] = torch.tensor(x['class'], dtype=torch.long)
    return {'id': identifiers, 'input_ids': input_ids, 'attention_mask': attention_mask, 'class_label': class_labels}

class BatchSampler:
    '''
    BatchSampler for variable length sequences, batching by similar lengths, to prevent excessive padding.
    '''
    def __init__(self, lengths, batch_size, mega_batch_size, drop_last = True):
        self.lengths = lengths
        self.batch_size = batch_size
        self.mega_batch_size = mega_batch_size
        self.drop_last = drop_last

    def __iter__(self):
        size = len(self.lengths)
        indices = list(range(size))
        random.shuffle(indices)

        step = self.mega_batch_size * self.batch_size
        for i in range(0, size, step):
            pool = indices[i:i+step]
            pool = sorted(pool, key=lambda x: self.lengths[x])
            mega_batch_indices = list(range(0, len(pool), self.batch_size))
            random.shuffle(mega_batch_indices) # shuffle the mega batches, so that the model doesn't see the same order of lengths every time. The small batch will however always be the one with longest lengths
            for j in mega_batch_indices:
                if self.drop_last and j + self.batch_size > len(pool): # drop the last batch if it's too small
                    continue
                batch = pool[j:j+self.batch_size]
                random.shuffle(batch) # shuffle the batch, so that the model doesn't see the same order of lengths every time
                yield batch

    def __len__(self):
        if self.drop_last:
            return len(self.lengths) // self.batch_size
        else:
            return (len(self.lengths) + self.batch_size - 1) // self.batch_size


In [8]:
import pickle

train_lengths_path = os.path.join(datasets_folder, 'train_lengths.pkl')
val_lengths_path = os.path.join(datasets_folder, 'val_lengths.pkl')
test_lengths_path = os.path.join(datasets_folder, 'test_lengths.pkl')

if os.path.exists(train_lengths_path):
    with open(train_lengths_path, 'rb') as f:
        train_lengths = pickle.load(f)
else:
    train_lengths = list(map(lambda x: len(x["input_ids"]), dataset_train))
    with open(train_lengths_path, 'wb') as f:
        pickle.dump(train_lengths, f)

if os.path.exists(val_lengths_path):
    with open(val_lengths_path, 'rb') as f:
        val_lengths = pickle.load(f)
else:
    val_lengths = list(map(lambda x: len(x["input_ids"]), dataset_val))
    with open(val_lengths_path, 'wb') as f:
        pickle.dump(val_lengths, f)

if os.path.exists(test_lengths_path):
    with open(test_lengths_path, 'rb') as f:
        test_lengths = pickle.load(f)
else:
    test_lengths = list(map(lambda x: len(x["input_ids"]), dataset_test))
    with open(test_lengths_path, 'wb') as f:
        pickle.dump(test_lengths, f)

In [9]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(dataset_test,
                            batch_sampler=BatchSampler(test_lengths,
                                                    config.batch_size,
                                                    config.mega_batch,
                                                    drop_last=False), 
                            collate_fn=collate_fn)
val_dataloader = DataLoader(dataset_val, 
                            batch_sampler=BatchSampler(val_lengths,
                                                    config.batch_size,
                                                    config.mega_batch,
                                                    drop_last=False),
                            collate_fn=collate_fn)
train_dataloader = DataLoader(dataset_train,
                            batch_sampler=BatchSampler(train_lengths,
                                                    config.batch_size,
                                                    config.mega_batch,
                                                    drop_last=False),
                            collate_fn=collate_fn)

In [10]:
from New1D.autoencoder_kl_1d import AutoencoderKL1D

model = AutoencoderKL1D(
    num_class_embeds=tokenizer.vocab_size + 1,  # the number of class embeddings
    
    down_block_types=(
        "DownEncoderBlock1D",  # a regular ResNet downsampling block
        "DownEncoderBlock1D",
        "DownEncoderBlock1D",
        "DownEncoderBlock1D",  # a ResNet downsampling block with spatial self-attention
    ),
    up_block_types=(
        "UpDecoderBlock1D",  # a ResNet upsampling block with spatial self-attention
        "UpDecoderBlock1D",
        "UpDecoderBlock1D",
        "UpDecoderBlock1D",  # a regular ResNet upsampling block
    ),
    block_out_channels=(128, 256, 512, 512),  # the number of output channels for each block
    mid_block_type="UNetMidBlock1D",  # the type of the middle block
    mid_block_channels=1024,  # the number of output channels for the middle block
    mid_block_add_attention=False,  # whether to add a spatial self-attention block to the middle block
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    transformer_layers_per_block=1, # how many transformer layers to use per ResNet layer. Not implemented yet.

    latent_channels=64,  # the dimensionality of the latent space

    num_attention_heads=1,  # the number of attention heads in the spatial self-attention blocks
    upsample_type="conv", # the type of upsampling to use, either 'conv' (and nearest neighbor) or 'conv_transpose'
    act_fn="gelu",  # the activation function to use
)

In [11]:
model.children
model.num_parameters()

60777560

In [12]:
sample_image = next(iter(val_dataloader))
print(sample_image.keys())
print(sample_image['id'][0])
print(sample_image['input_ids'][0])
print(sample_image['attention_mask'][0])
print(sample_image['class_label'][0])

dict_keys(['id', 'input_ids', 'attention_mask', 'class_label'])
UniRef50_A0A1J3G3G8
tensor([ 0,  3, 11, 12,  7, 18, 18, 17,  8,  3, 11, 18, 19, 12, 12, 19, 19, 17,
        18, 14,  9,  6,  7, 12,  5, 11, 17, 10,  6,  3, 10, 11, 14, 16, 14, 15,
         5, 12,  6, 10,  8, 10, 11, 10,  7,  5,  7, 15,  4, 20,  6, 12,  8, 12,
        15,  6,  8,  4,  5,  9, 20,  5,  7, 10, 10, 18, 22, 15, 11, 15,  5,  3,
         8,  5, 12, 12, 12, 11, 12,  7, 13, 18, 19, 11, 22, 13, 14, 16, 16, 12,
         6, 18,  7, 10,  6, 19, 19, 11, 15, 19,  4, 12, 20,  3,  5, 13,  7,  7,
        19, 21,  3, 19,  6, 18,  3, 16, 11,  7,  8, 20, 15, 17, 12, 20,  7,  9,
         8, 19, 18,  7,  7,  1, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.,

In [13]:
import torch

class_labels = sample_image['class_label'][0].unsqueeze(0)
attention_mask = sample_image['attention_mask'][0].unsqueeze(0)
print(class_labels.shape)
print(attention_mask.shape)

torch.Size([1])
torch.Size([1, 144])


In [14]:
import torch.nn.functional as F
from typing import Tuple
from New1D.vae import AutoencoderKLOutput1D

input_ids = sample_image['input_ids']
attention_mask = sample_image['attention_mask']

output = model(sample = input_ids,
                attention_mask = attention_mask,
                sample_posterior = True, # Should be set to true in training
)

def loss_fn(output: AutoencoderKLOutput1D, input_ids: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
    ce_loss = F.cross_entropy(output.sample, input_ids, reduction='none')
    ce_loss = torch.sum(
        ce_loss * output.attention_masks[0]
    ) / output.attention_masks[0].sum()
    
    kl_loss = output.latent_dist.kl()
    kl_loss = torch.sum(
        kl_loss * output.attention_masks[-1]
    ) / output.attention_masks[-1].sum()

    return ce_loss, kl_loss

# loss = loss_fn(output, input_ids)
# print(loss)

In [15]:
from diffusers.optimization import get_cosine_schedule_with_warmup

if config.optimizer == "Adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
elif config.optimizer == "AdamW":
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
elif config.optimizer == "Adamax":
    optimizer = torch.optim.Adamax(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
elif config.optimizer == "SGD":
    optimizer = torch.optim.SGD(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay, momentum=config.SGDmomentum)
else:
    raise ValueError("Invalid optimizer, choose between `AdamW`, `Adam`, `SGD`, and `Adamax`")

lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs // config.gradient_accumulation_steps),
)

In [16]:
import os

from typing import Optional

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

@torch.no_grad()
def evaluate(config, name: str, output, dataloader, output_dir: str
    ) -> float:

    if output_dir is not None:
        test_dir = os.path.join(config.output_dir, "samples")
        os.makedirs(test_dir, exist_ok=True)

    running_loss = 0.0
    num_correct_residues = 0
    total_residues = 0

    for i, sample in enumerate(dataloader):

        output = model(sample = sample['input_ids'],
                            attention_mask = sample['attention_mask'],
                            sample_posterior = False, # Should be set to true in training
        )

        ce_loss, kl_loss = loss_fn(output, sample['input_ids'])
        loss = ce_loss + kl_loss * config.kl_weight
        running_loss += loss.item()

        token_ids_pred = logits_to_token_ids(tokenizer, output.sample, cutoff = config.cutoff)

        token_ids_correct = ((sample['input_ids'] == token_ids_pred) & (sample['attention_mask'] == 1)).long()
        num_residues = torch.sum(sample['attention_mask'], dim=1).long()

        num_correct_residues += token_ids_correct.sum().item()
        total_residues += num_residues.sum().item()

        # Decode the predicted sequences, and remove zero padding
        seqs_pred = tokenizer.batch_decode(token_ids_pred, skip_special_tokens=config.skip_special_tokens)
        seqs_lens = torch.sum(sample['attention_mask'], dim=1).long()
        seqs_pred = [seq[:i] for seq, i in zip(seqs_pred, seqs_lens)]

        # Save all samples as a FASTA file
        seq_record_list = [SeqRecord(Seq(seq), id=str(sample['id'][i]), 
                        description=
                        f"classlabel: {sample['class_label'][i].item()} acc: {token_ids_correct[i].sum().item() / num_residues[i].item():.2f}")
                        for i, seq in enumerate(seqs_pred)]
        with open(f"{test_dir}/{name}.fa", "a") as f:
            SeqIO.write(seq_record_list, f, "fasta")
    
    acc = num_correct_residues / total_residues
    print(f"{name}, val_loss: {running_loss / len(dataloader):.4f}, val_accuracy: {acc:.4f}")
    logs = {"val_loss": loss.detach().item(), 
            "val_ce_loss": ce_loss.detach().item(), 
            "val_kl_loss": kl_loss.detach().item(),
            "val_acc": acc,
            }
    return logs

In [17]:
from accelerate import Accelerator
from huggingface_hub import create_repo, upload_folder
from tqdm.auto import tqdm
from pathlib import Path
import os

from grokfast import gradfilter_ema

def train_loop(config, model, optimizer, train_dataloader, test_dataloader, lr_scheduler):
    grads = None
    
    # Initialize accelerator and tensorboard logging
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )
    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        if config.push_to_hub:
            repo_id = create_repo(
                repo_id=config.hub_model_id or Path(config.output_dir).name, exist_ok=True
            ).repo_id
        accelerator.init_trackers("train_example", config=vars(config))

    # Prepare everything
    # There is no specific order to remember, you just need to unpack the
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, test_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, test_dataloader, lr_scheduler
    )

    global_step = 0
    val_loss = float("inf")

    # Now you train the model
    model.train()
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):

            with accelerator.accumulate(model):
                input = batch['input_ids']
                attention_mask = batch['attention_mask']
                # Predict the noise residual
                output = model(sample = input,
                                attention_mask = attention_mask,
                                sample_posterior = True, # Should be set to true in training
                )
                ce_loss, kl_loss = loss_fn(output, input)
                loss = ce_loss + kl_loss * config.kl_weight
                accelerator.backward(loss)

                if config.grokfast:
                    grads = gradfilter_ema(model, grads=grads, alpha=config.grokfast_alpha, lamb=config.grokfast_lamb) 

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"train_loss": loss.detach().item(), 
                    "train_ce_loss": ce_loss.detach().item(), 
                    "train_kl_loss": kl_loss.detach().item(), 
                    "lr": lr_scheduler.get_last_lr()[0], 
                    "step": global_step,
            }
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

            if global_step == 1 or global_step % config.save_image_steps == 0 or global_step == len(train_dataloader):
                
                name = f"step_{global_step//1000:04d}k"

                model.eval() # Set model to eval mode to generate images
                logs = evaluate(config, name, model, test_dataloader, output_dir = config.output_dir)
                accelerator.log(logs, step=global_step)

                new_val_loss = logs["val_loss"]

                if new_val_loss < val_loss: # Save the model if the validation loss is lower
                    val_loss = new_val_loss
                    if config.push_to_hub:
                        upload_folder(
                            repo_id=repo_id,
                            folder_path=config.output_dir,
                            commit_message=f"Steps {global_step}",
                            ignore_patterns=["step_*", "epoch_*"],
                        )
                    else:
                        model.save_pretrained(config.output_dir)
                model.train() # Set model back to train mode

In [18]:
from accelerate import notebook_launcher

args = (config, model, optimizer, train_dataloader, test_dataloader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)


Launching training on one GPU.


/home/kkj/ProtDiffusion/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/923922 [00:00<?, ?it/s]

step_0000k, val_loss: 3.1721, val_accuracy: 0.0563
step_0010k, val_loss: 1.6443, val_accuracy: 0.4222
step_0020k, val_loss: 1.3537, val_accuracy: 0.5203
step_0030k, val_loss: 1.2636, val_accuracy: 0.5476
step_0040k, val_loss: 1.2409, val_accuracy: 0.5585
step_0050k, val_loss: 1.1480, val_accuracy: 0.5900
step_0060k, val_loss: 1.0489, val_accuracy: 0.6329
step_0070k, val_loss: 1.0271, val_accuracy: 0.6373
step_0080k, val_loss: 1.0123, val_accuracy: 0.6411
step_0090k, val_loss: 1.0792, val_accuracy: 0.6105
step_0100k, val_loss: 1.0763, val_accuracy: 0.6133
step_0110k, val_loss: 1.2455, val_accuracy: 0.5695
step_0120k, val_loss: 0.9766, val_accuracy: 0.6526
step_0130k, val_loss: 0.9838, val_accuracy: 0.6491
step_0140k, val_loss: 0.9753, val_accuracy: 0.6507
step_0150k, val_loss: 0.9667, val_accuracy: 0.6521
step_0160k, val_loss: 0.8793, val_accuracy: 0.6895
step_0170k, val_loss: 0.9040, val_accuracy: 0.6864
step_0180k, val_loss: 1.2841, val_accuracy: 0.5385
step_0190k, val_loss: 0.7475, v

  0%|          | 0/923922 [00:00<?, ?it/s]

step_0930k, val_loss: 0.4153, val_accuracy: 0.8626
step_0940k, val_loss: 0.4349, val_accuracy: 0.8551
step_0950k, val_loss: 0.4327, val_accuracy: 0.8555
step_0960k, val_loss: 0.4115, val_accuracy: 0.8641
step_0970k, val_loss: 0.4148, val_accuracy: 0.8630
step_0980k, val_loss: 0.3872, val_accuracy: 0.8742
step_0990k, val_loss: 0.4066, val_accuracy: 0.8664
step_1000k, val_loss: 0.3878, val_accuracy: 0.8734
step_1010k, val_loss: 0.3897, val_accuracy: 0.8739
step_1020k, val_loss: 0.4039, val_accuracy: 0.8670
step_1030k, val_loss: 0.3933, val_accuracy: 0.8708
step_1040k, val_loss: 0.3985, val_accuracy: 0.8690
step_1050k, val_loss: 0.3859, val_accuracy: 0.8742


In [ ]:
for i, para in enumerate(model.parameters()):
    print(f'{i + 1}th parameter tensor:', para.shape)
    print(para)
    print(para.grad)